## ЗАДАЧА №1

### SQL

In [ ]:
DROP TABLE IF EXISTS t1;

CREATE TABLE IF NOT EXISTS t1 (
    k INT,
    id INT,
    phone VARCHAR(12),
    mail VARCHAR(50)
);

INSERT INTO t1 (k, id, phone, mail) VALUES
  (1, 12345, '89997776655', 'test@mail.ru'),
  (2, 54321, '87778885566', 'two@mail.ru'),
  (3, 98765, '87776664577', 'three@mail'),
  (4, 66678, '87778885566', 'four@mail.ru'),
  (5, 34567, '84547895566', 'four@mail.ru'),
  (6, 34567, '89087545678',  'five@mail.ru');
 
 /*
 Я создал процедуру my_procedure, которая вытаскивает связанные значения из таблицы
 Всё что нужно сделать - вызвать её и передать два аргумента:
 1. Признак, по которому собираемся искать связанные значения
 2. Значение признака
 */
 
DROP PROCEDURE IF EXISTS my_procedure;
 
 DELIMITER $$
CREATE PROCEDURE my_procedure(IN col VARCHAR(50), val VARCHAR(50))
BEGIN
    SET @c = CONCAT('
		WITH RECURSIVE my_table AS (
		SELECT k, id, phone, mail
		FROM t1
		WHERE ', col, ' = ', val, '
		UNION
		SELECT t1.k, t1.id, t1.phone, t1.mail
		FROM t1
		JOIN my_table ON t1.id = my_table.id 
		OR t1.phone = my_table.phone 
		OR t1.mail = my_table.mail
)
SELECT DISTINCT * FROM my_table;
    ');
    PREPARE stmt FROM @c;
    EXECUTE stmt;
    DEALLOCATE PREPARE stmt;
END$$
DELIMITER ; 

CALL my_procedure('phone', '87778885566')

### Python

In [ ]:
import pandas as pd
import warnings

warnings.simplefilter(action='ignore', category=Warning)

def my_func(df, col, val):
    result = pd.DataFrame(columns=df.columns)
    passed = set() #Сюда будем складывать индексы тех строк, по которым мы уже прошлись
    stack = df[df[col] == val].index.tolist() #Сюда складываем индексы подходящих нам строк
    while stack != []:
        cur_index = stack.pop()
        if cur_index not in passed:
            passed.add(cur_index)
            result = result.append(df.loc[cur_index])
            related_indices = df[
                (df['id'] == df.loc[cur_index, 'id']) | 
                (df['phone'] == df.loc[cur_index, 'phone']) | 
                (df['mail'] == df.loc[cur_index, 'mail'])
            ].index.tolist()
            stack.extend(related_indices)
    return result

data = {'key': [1, 2, 3, 4, 5, 6],
        'id': [12345, 54321, 98765, 66678, 34567, 34567],
        'phone': ['89997776655', '87778885566', '87776664577', '87778885566', '84547895566', '89087545678'],
        'mail': ['test@mail.ru', 'two@mail.ru', 'three@mail', 'four@mail.ru', 'four@mail.ru', 'five@mail.ru']}

df = pd.DataFrame(data)

col = input() #Признак, по которому ищем связанные данные
val = input() #Значение признака
val = type(df.loc[0, col])(val) #Эта строчка нужна чтобы перевести тип данных val в тот который соответствует нужному
#признаку для дальнейшего сравнения, потому как значения переменных поступают в виде строки, но при этом в данных есть столбец
#id, который обладает типом int, поэтому в случае когда признаком выступает id, поиск будет совершён некорректно, если не 
#согласовать типы данных

my_func(df, col, val)

## Задача №2

### SQL

In [ ]:
drop table if EXISTS LOANS_TABLE;
CREATE TABLE if not exists LOANS_TABLE (
LOAN_ID int,
CLIENT_ID int,
LOAN_DATE date,
LOAN_AMOUNT float);

drop table if EXISTS CLIENTS_TABLE;
CREATE TABLE if not exists CLIENTS_TABLE (
CLIENT_ID int,
CLIENT_NAME NVARCHAR(20),
BIRTHDAY date,
GENDER NVARCHAR(20));

INSERT INTO CLIENTS_TABLE
VALUES
(1, 'bob', '20200115', 'male'),
(2, 'rocky', '20200215', 'female'),
(3, 'like', '20200215', 'female'),
(4, 'ricky', '20200215', 'male');

INSERT INTO LOANS_TABLE
VALUES
(1, 1, '20200115', 10000),
(2, 2, '20200215', 20000),
(3, 3, '20200315', 30000),
(4, 4, '20200415', 40000),
(5, 1, '20200116', 15000),
(6, 2, '20200315', 35000),
(7, 3, '20200315', 5000),
(8, 1, '20200115', 1500),
(9, 2, '20200115', 500),
(10, 1, '20200115', 1500);


/* Я создал процедуру, которая при вызове выводит количество договоров всех номеров вплоть до самого максимального.*/


DROP PROCEDURE IF EXISTS my_procedure_2; 


 DELIMITER $$
CREATE PROCEDURE my_procedure_2()
BEGIN
	CREATE TEMPORARY TABLE IF NOT EXISTS my_table1 AS (
    SELECT gender, loan_date,
    ROW_NUMBER() OVER (PARTITION BY clients_table.client_id ORDER BY loans_table.loan_date ASC) AS contract_number
    FROM CLIENTS_TABLE
    JOIN LOANS_TABLE ON clients_table.client_id = loans_table.client_id
    );
	SET @num := 1;
    SET @str := '';
	WHILE @num <= (SELECT MAX(contract_number) FROM my_table1) DO
		SET @str = CONCAT(@str, ', SUM(CASE contract_number WHEN ', @num, 
        ' THEN 1 ELSE 0 END) AS count_', CAST(@num AS CHAR));
        SET @num = @num + 1;
	END WHILE;
    SET @c = CONCAT('SELECT gender', @str, ' FROM my_table1 
    WHERE YEAR(loan_date) = 2020
    GROUP BY gender');
    PREPARE stmt FROM @c;
    EXECUTE stmt;
    DEALLOCATE PREPARE stmt;
    DROP TEMPORARY TABLE IF EXISTS my_table1;
END$$
DELIMITER ; 

CALL my_procedure_2();